In [1]:
import csv
import tensorflow as tf
import numpy as np
import urllib

physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass


In [2]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
    urllib.request.urlretrieve(url, 'sunspots.csv')

    time_step = []
    sunspots = []

    with open('sunspots.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            sunspots.append(float(row[2]))
            time_step.append(int(row[0]))

    series = np.array(sunspots)

    min = np.min(series)
    max = np.max(series)
    series -= min
    series /= max
    time = np.array(time_step)


    split_time = 3000
    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]


    window_size = 30
    batch_size = 32
    shuffle_buffer_size = 1000

    train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size,
                                 shuffle_buffer=shuffle_buffer_size)

    print(np.array(train_set).shape)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=60, kernel_size=5,
                               strides=1, padding="causal",
                               activation="relu",
                               input_shape=[None, 1]),
        tf.keras.layers.LSTM(60, return_sequences=True),
        tf.keras.layers.LSTM(60, return_sequences=True),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1)
    ])

    model.compile(loss='mae', optimizer='adam')
    model.fit(train_set, batch_size=256, epochs=100)

    return model



if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")



()
Epoch 1/100
93/93 [==============================] - 16s 12ms/step - loss: 0.0750
Epoch 2/100
93/93 [==============================] - 1s 8ms/step - loss: 0.0509
Epoch 3/100
93/93 [==============================] - 1s 8ms/step - loss: 0.0474
Epoch 4/100
93/93 [==============================] - 1s 8ms/step - loss: 0.0459
Epoch 5/100
93/93 [==============================] - 1s 8ms/step - loss: 0.0454
Epoch 6/100
93/93 [==============================] - 1s 8ms/step - loss: 0.0453
Epoch 7/100
93/93 [==============================] - 1s 8ms/step - loss: 0.0450
Epoch 8/100
93/93 [==============================] - 1s 8ms/step - loss: 0.0446
Epoch 9/100
93/93 [==============================] - 1s 9ms/step - loss: 0.0446
Epoch 10/100
93/93 [==============================] - 1s 8ms/step - loss: 0.0447
Epoch 11/100
93/93 [==============================] - 1s 8ms/step - loss: 0.0445
Epoch 12/100
93/93 [==============================] - 1s 8ms/step - loss: 0.0447
Epoch 13/100
93/93 [============